# Homework for week 2

## Question 1: [IPO] Withdrawn IPOs by Company Type

In [27]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from io import StringIO

import string

In [5]:
url = "https://stockanalysis.com/ipos/withdrawn/"
response = requests.get(url)
#response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
html_io = StringIO(response.text)
df_ipo = pd.read_html(html_io)

In [10]:
df_ipo = df_ipo[0]

In [11]:
df_ipo.shape

(100, 4)

In [12]:
df_ipo.head()

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [28]:
def company_class(name):

     # Convert to lowercase and remove punctuation
    name = name.lower()
    name = name.translate(str.maketrans('', '', string.punctuation))
    words = name.split()

    if 'acquisition' in words and ('corp' in words or 'corporation' in words):
        return 'Acq.Corp'
    elif 'inc' in words or 'incorporated' in words :
        return 'Inc'
    elif 'group' in words:
        return 'Group'
    elif 'holdings' in words:
        return 'Holdings'
    elif 'ltd' in words or 'limited' in words:
        return 'Ltd'
    else:
        return 'Other'

In [29]:
df_ipo["Company Class"] = df_ipo["Company Name"].apply(company_class)


In [32]:
df_ipo['Company Class'].value_counts()

Company Class
Inc         51
Acq.Corp    21
Ltd         12
Other        6
Holdings     6
Group        4
Name: count, dtype: int64